<img src="images/logo_city.png" align="right" width="20%">

# Training A Graph State Prediction Network (GSPNet)

This notebook is a prototype on how to train a graph state prediction network using CNN and LSTM.
The content is devided into __3__ parts:

1. Data preprocessing
2. Model building, Training and Tuning
3. Prediction and per demand modifacation

The model is built with [PyTorch](https://pytorch.org/).

## Part 1: Data Preprocessing

### 1. Import necessary libraries:

We will use pandas and numpy as main data examine and cleansing util and use dask to handle parallel computing cases

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlalchemy
import dask
import time
import psycopg2
import warnings
import re
import torch
from PIL import Image
from IPython.display import display
from scipy import stats
from matplotlib import pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


<br></br>
<br></br>
We'll first, use a small sample data (approximately 1 million rows) and see a few rows of it:

In [2]:
# Load already cleaned data sample using plain pandas
t1 = time.time()
table = pd.read_csv('dataset/nytaxi_yellow_2017_mar.csv')
print(f'Load time is {time.time() - t1 :2f}.')
table.head()

Load time is 42.912627.


,tripid,tpep_pickup_datetime,tpep_dropoff_datetime,pulocationid,dolocationid,trip_distance,passenger_count,total_amount,trip_time,trip_avg_speed,trip_time_sec
0,35972297,2017-03-09 21:30:11,2017-03-09 21:44:20,148,48,4.06,1,18.36,00:14:09,17.215548,849
1,35972298,2017-03-09 21:47:00,2017-03-09 21:58:01,48,107,2.73,1,12.80,00:11:01,14.868381,661
2,35972299,2017-03-09 22:01:08,2017-03-09 22:11:16,79,162,2.27,1,14.12,00:10:08,13.440789,608
3,35972300,2017-03-09 22:16:05,2017-03-10 06:26:11,237,41,3.86,1,17.29,08:10:06,0.472557,29406
4,35972301,2017-03-31 06:31:53,2017-03-31 06:41:48,41,162,3.45,1,13.30,00:09:55,20.873950,595


### 2. First sight of the data

Let's see the the type of columns and some basic statistics of the sample data:

In [ ]:
# column data types
table.dtypes

In [ ]:
# rows and columns
table.shape

In [ ]:
# basic statistics
table.describe()

The columns we will use are: **tpep_pickup_datetime, pulocationid, tpep_dropoff_datetime, dolocationid.**
These information are used to generate graphs that are used to discribe the general traffic state of New York.

Notice currently the **tpep_pickup_datetime** and **tpep_dropoff_datetime** are of type `object`:

In [ ]:
table.loc[:,['tpep_pickup_datetime', 'tpep_dropoff_datetime']].head()

In order to do **time slice** later, these two feilds must be sorted in chronological mananer, first by `tpep_pickup_datetime` and then by `tpep_dropoff_datetime`, as follows:

In [ ]:
# pick first 10 rows
# not supported for dask DataFrame
table.sort_values(by=['tpep_pickup_datetime', 'tpep_dropoff_datetime']).head(10)

We can see from above that the taxi trips are nicely sorted according to **time**. What we need to do is convert this tabular date into graphs that described by **adjacency matrice**.

First, let's see what are `pulocationid` and `dolocationid`:

<img src="images/map.jpg" align="center" width="60%">

They are code number for a district in Manhattan, New York. We will mainly describe the traffic state and prediction at **this** level.

### 3. Time slicing

Now we've done a very superficial exploration of the data. It's time to convert the tabular data to graphs that are to be fed into our neural network model.

The first problem is, **how to decide a feasible time interval?**

Of course, the time interval should be a varaible that is to be selected **per need. However,** it is a reasonable thinking to let a time interval (i.e. a **snapshot**) to contain as many as **complete trip** as possible. See the diagram below:

<img src="images/snapshot.png" align="center" width="70%">

#### **If the time interval is too small**:

The majority of the matrices will be very sparse or even completely blank.

#### If the time interval is too big:

Then information about trips will be densely squeezed into one matrix and a lot of details on changes along time will be lost.

Thus, it is important to choose time interval wisely. Let's look at the data again:

In [ ]:
table.loc[:,['trip_distance', 'trip_time_sec', 'trip_avg_speed']].describe()

The an average trip lasts for roughly 900 seconds, which is **15 minutes**.

The median(i.e. 50% percentile) is roughly 600 seconds, which is **10 minutes**.

Let's plot the distribution of column `trip_time_sec` to have a better intuition:

In [ ]:
warnings.simplefilter('ignore')
sns.set(color_codes=True)
sns.distplot(table.loc[:,'trip_time_sec']);

It seems that the trips are mostly concentrated in a shorter time range, while some outliers may significantly biased the average.

Let's see how how many trips are longer than 30 minutes (1800 secs):

In [ ]:
# rows with column 'trip_time_sec' value larger than 1800 divided by total number of rows
table.loc[table['trip_time_sec'] > 1800].shape[0] / table.shape[0]

Dask version of above cell:

There are **95%** of the trips last shorter than 30 minutes, let's plot distribution at this range:

In [ ]:
# distribution of 95% trip data in time duration
temp = table.loc[table['trip_time_sec'] <= 1800]
sns.distplot(temp.loc[:,'trip_time_sec']);

In [ ]:
# statistics on the 95% data
temp.loc[:,['trip_distance', 'trip_time_sec', 'trip_avg_speed']].describe()

In [ ]:
# value and their appearance
pd.DataFrame(temp.loc[:,'trip_time_sec'].value_counts().sort_values())

More than 50% of the trips have trip time between 6 to 15 minutes, with a median of 10 minutes.

#### As a result, we primarily decide to use 10 minutes as time interval.

### 4. From tabular data to graph

Since we have decided a time interval, it's time to generate matrices. Basically, the idea is shown as figure below:

<img src="images/matrices.png" align="left" width="85%">

Apart from **outgoin** and **incoming** matrices shown above, an additional **Domestic** matrix is add because 50% of trips last less than the **chosen interval 10 minutes**. Such matrices can be combined into a **Tensor** illustrated as follows:

<img src="images/layers.png" align="center" width="35%">

#### STRICT DEFINITION OF OID layers

+ **F**uture layer
    Condition: 

    1. left bound of interval  <= tpep_pickup_datetime  < right bound of interval
    2. right bound of interval <= tpep_dropoff_datetime


+ **P**ast layer
    Condition: 

    1. left bound of interval  <= tpep_dropoff_datetime  < right bound of interval
    2. tpep_pickup_datetime < left bound of interval


+ **N**ow layer
    Condition: 

    1. left bound of interval  <= tpep_pickup_datetime < tpep_dropoff_datetime  < right bound of interval

### Notice: The PNF layers are redefined and renamed from OID layers.

#### Extract relevant columns:

To generate tensor defined above, **four** key columns are needed.They are `tpep_pickup_datetime`, `tpep_dropoff_datetime`, `pulocationid` and `dolocationid`.

We first extract them from original table:

In [ ]:
# draw four needed columns and a id column, then sort according to time.
# the `tripid` column is for the sake of naming.
tensor_gen_f = table.loc[:,['tripid',
                          'tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'pulocationid', 'dolocationid']
                      ].sort_values(by=['tpep_pickup_datetime', 'tpep_dropoff_datetime']) # the first sort condition rules
tensor_gen_f.head()

#### Slice the table to sub-tables according to time interval (10 min)

Since the columns are extracted, now it's time to generate tensors.

The dataset is trip data of all yellow taxi cabs in Manhattan area in **January**, 2017.As we are making 10-minute-splices, there will be total:

$$
6 \times 24 \times 31 = 4464 \space slices(i.e. \space snapshots)
$$

Each slice has 3 layers, that is in total

$$
4464 \times 3 = 13392 \space matrices
$$
to be generated. 

First, let's try to generate one image (i.e. a *tensor* with **3** layers).

The time columns need to be converted to pandas timestamp type in order to compare:

In [ ]:
# current type is python str
type(tensor_gen_f.iloc[0,1])

In [ ]:
# convert to timestamp
pd.to_datetime(tensor_gen_f.iloc[0,1])

In [ ]:
t1 = pd.to_datetime(tensor_gen_f.iloc[0,1]) # 2017-01-01 00:00:00
t2 = pd.to_datetime(tensor_gen_f.iloc[0,2]) # 2017-01-01 00:01:00
t1 < t2 # test if can compare time

In [ ]:
# apply type cast to all values:
tensor_gen_f['tpep_pickup_datetime'] = pd.to_datetime(tensor_gen_f['tpep_pickup_datetime'])
tensor_gen_f['tpep_dropoff_datetime'] = pd.to_datetime(tensor_gen_f['tpep_dropoff_datetime'])

tensor_gen_f.dtypes

In [ ]:
tensor_gen_f.head()

The interval **boundaries** are fixed when interval is set.

In [ ]:
# create intervals
intervals = pd.date_range('2017-04-01 00:00:00', '2017-05-01 00:00:00', freq='10min')
print(len(intervals))
intervals

In [ ]:
type(intervals)

**Notice**:

When slicing time interval in such manner, we are assuming the **entire** month is monitored. Under this setting, we do not consider another day as a 'fresh start'.



In [ ]:
str(intervals[1])

In [ ]:
# First sub table:
first = tensor_gen_f.loc[tensor_gen_f['tpep_pickup_datetime'] < intervals[1]]
print(f'shape is: {first.shape}')
first.head()

In [ ]:
# sort to have a clearer look:
first.sort_values(by=['pulocationid', 'dolocationid']).head(20)

The `first` will generate Outgoing, Incoming and Domestic layers, that is, 3 matrices of size: 

$$
(number \space of\space zones)\space \times \space (number\space of\space zones)
$$

Let's see the zone numbers:

In [ ]:
# Load zone lookup table
zones = pd.read_csv('dataset/taxi_zone_lookup.csv')
print(f'shape is: {zones.shape}')

Total 265 zones, indexing from 0 to 264.

Among which, we only care about `Yellow Zone` or `Manhattan` area:

In [ ]:
y_zone = zones.loc[zones['Borough'] == 'Manhattan']
print(y_zone.shape)
img_size = y_zone.shape[0]

Thus, the layers should be matrices of size (69, 69), and each timeslice(snapshot) should be tensors of size (69, 69, 3).

Create the first tensor:

### The key is to use generated intervals to 'catch' trips.

In [ ]:
# legacy code, currently in use
left = 1481
right = 1482

# convert dtype for entire table here:
table['tpep_pickup_datetime'] = pd.to_datetime(table['tpep_pickup_datetime'])
table['tpep_dropoff_datetime'] = pd.to_datetime(table['tpep_dropoff_datetime'])

sorted_table = table.loc[:,['tripid',
                          'tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'pulocationid', 'dolocationid']
                        ].sort_values(by=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

# The condition of making snapshot should be: at least one temporal end of a trip should be within the bounds:
snap = sorted_table.loc[((sorted_table['tpep_pickup_datetime'] >= intervals[left]) &
                        (sorted_table['tpep_pickup_datetime'] < intervals[right])) |
                        ((sorted_table['tpep_dropoff_datetime'] >= intervals[left]) &
                        (sorted_table['tpep_dropoff_datetime'] < intervals[right]))
                        ]
print(f'sorted_table.shape -> {sorted_table.shape}')
print(f'snap.shape -> {snap.shape}')

# temp table to generate F,P,N layers
# No need to sort
temp_snap = snap.loc[:,['tripid',
                          'tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'pulocationid', 'dolocationid']
                    ]


# Use the interval to 'catch' trips.
f_flayer = temp_snap.loc[(temp_snap['tpep_pickup_datetime'] < intervals[right]) &
                            (temp_snap['tpep_pickup_datetime'] >= intervals[left]) &
                            (temp_snap['tpep_dropoff_datetime'] >= intervals[right])
                           ]

f_player = temp_snap.loc[(temp_snap['tpep_pickup_datetime'] < intervals[left]) &
                            (temp_snap['tpep_dropoff_datetime'] >= intervals[left]) &
                            (temp_snap['tpep_dropoff_datetime'] < intervals[right])
                           ]

f_nlayer = temp_snap.loc[(temp_snap['tpep_pickup_datetime'] >= intervals[left]) &
                            (temp_snap['tpep_dropoff_datetime'] < intervals[right])
                           ]

print(f'f_flayer.shape: {f_flayer.shape}')
print(f'f_player.shape: {f_player.shape}') # There will be no incoming trips for the first snapshot.
print(f'f_nlayer.shape: {f_nlayer.shape}')

# Their shape should be the same
assert temp_snap.shape[0] == f_flayer.shape[0] + f_player.shape[0] + f_nlayer.shape[0]

### Wrap the above procedure into a function for later use:

In [ ]:
def gen_snap_layers(table, bound):
    '''
    Generate Past layer, Now layer and Future layer for one snapshot.
    Params:
        table:
        bounds:
    Return:
        PNF layers, a list.
    '''
    # left bound and right bound of time interval
    assert type(bound) == tuple
    left = bound[0]
    right = bound[1]
    
    print(left, right)

    sorted_table = table.loc[:, ['tripid',
                                 'tpep_pickup_datetime',
                                 'tpep_dropoff_datetime',
                                 'pulocationid',
                                 'dolocationid']].sort_values(by=['tpep_pickup_datetime',
                                                                  'tpep_dropoff_datetime'])

    # The condition of making snapshot should be:
    # at least one temporal end of a trip should be within the bounds:
    snap = sorted_table.loc[
        ((sorted_table['tpep_pickup_datetime'] >= left) &
         (sorted_table['tpep_pickup_datetime'] < right)) |
        ((sorted_table['tpep_dropoff_datetime'] >= left) &
         (sorted_table['tpep_dropoff_datetime'] < right))]

    # temp table to generate F,P,N layers
    # keep snap intact
    temp_snap = snap.loc[:, ['tripid',
                             'tpep_pickup_datetime',
                             'tpep_dropoff_datetime',
                             'pulocationid',
                             'dolocationid']]

    # Use the interval to 'catch' corresponding trips.
    # future layer
    f_layer = temp_snap.loc[(temp_snap['tpep_pickup_datetime'] < right) &
                             (temp_snap['tpep_pickup_datetime'] >= left) &
                             (temp_snap['tpep_dropoff_datetime'] >= right)]
    # past layer
    p_layer = temp_snap.loc[(temp_snap['tpep_pickup_datetime'] < left) &
                             (temp_snap['tpep_dropoff_datetime'] >= left) &
                             (temp_snap['tpep_dropoff_datetime'] < right)]
    # now layer
    n_layer = temp_snap.loc[(temp_snap['tpep_pickup_datetime'] >= left) &
                             (temp_snap['tpep_dropoff_datetime'] < right)]

    # Their count should add up to total trips caught
    assert temp_snap.shape[0] == f_layer.shape[0] + p_layer.shape[0] + n_layer.shape[0]

    return p_layer, n_layer, f_layer

Sanity check:

In [ ]:
# Check if the trips add up:
print(snap.shape)
print(f_flayer.shape[0] + f_player.shape[0] + f_nlayer.shape[0])
assert snap.shape[0] == f_flayer.shape[0] + f_player.shape[0] + f_nlayer.shape[0]
f_flayer.head(20)

Each pair of (pulocationid - 1, dolocationid - 1) indicates adding one to the snapshot:

### 5. Generating Image

Before generating image, we have to map the zone id to a range from 0 to 54:

In [ ]:
# very important globals:
real_id = list(map(str, list(y_zone.loc[:,'LocationID'])))
conv_id = [i for i in range(img_size)]
assert len(real_id) == len(conv_id)
mp = dict(zip(real_id, conv_id))
# the line below is a dirty fix, be causious in future!

In [ ]:
# create a snapshot:
first_snapshot = np.zeros([img_size, img_size, 3], dtype='float64')
print(first_snapshot.shape)
print(first_snapshot[1,2,1])

left_zones = set()

# future-Red: 0  
for _, row in f_flayer.iterrows():
    try:
        first_snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 0] += 1 # inplace increment 1 in numpy
    except Exception as e:
        left_zones.add(str(row['pulocationid']))
        left_zones.add(str(row['dolocationid']))

# past-Green: 1
for _, row in f_player.iterrows():
    try:
        first_snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 1] += 1
    except Exception as e:
        left_zones.add(str(row['pulocationid']))
        left_zones.add(str(row['dolocationid']))

# now-Blue: 2        
for _, row in f_nlayer.iterrows():
    try:
        first_snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 2] += 1
    except Exception as e:
        left_zones.add(str(row['pulocationid']))
        left_zones.add(str(row['dolocationid']))

print(f'left_zones: {left_zones}')
print(f'left_zones length: {len(left_zones)}')

print(f'F max -> {first_snapshot[:,:,0].max()}')
print(f'P max -> {first_snapshot[:,:,1].max()}')
print(f'N max -> {first_snapshot[:,:,2].max()}')

# convert numpy array to a image:
# tb = pd.DataFrame(first_snapshot[:,:,0])
first_snapshot *= 255//first_snapshot.max()

In [ ]:
# Plot FPN layer values:
temp_f = np.reshape(first_snapshot[:,:,0], (1,-1))
temp_p = np.reshape(first_snapshot[:,:,1], (1,-1))
temp_n = np.reshape(first_snapshot[:,:,2], (1,-1))

In [ ]:
sns.distplot(temp_f, axlabel='Future (Red)', color='red');

In [ ]:
sns.distplot(temp_p, axlabel='Past (Green)', color='green');

In [ ]:
sns.distplot(temp_n, axlabel='Now (Blue)', color='blue');

In [ ]:
first_snapshot = first_snapshot.astype('uint8')
first_image = Image.fromarray(first_snapshot)
first_image.resize((690,690))

In [ ]:
# image save directory:
img_path = r'D:\GITHUB\GSPNet\snapshots'

# save image
# first_image.save(f'{img_path}\\1.jpg')
# first_image.resize((690,690)) # multiply by factor of 100

### Put above codes into a function:

In [ ]:
# Function that combines layers to an image
def gen_image(p_layer, n_layer, f_layer):
    '''
    Generate an image using given matrices.
    Params:
        p_layer: matrix of past layer
        n_layer: matrix of now layer
        f_layer: matrix of future layer
    Return:
        A PIL image.
    '''
    # create a snapshot
    snapshot = np.zeros([img_size, img_size, 3], dtype='float64')

    # unexpected zones
    left_zones = set()

    # future-Red: 0
    for _, row in f_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 0] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # past-Green: 1
    for _, row in p_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 1] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # now-Blue: 2
    for _, row in n_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 2] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # normalize
    snapshot *= 255 // snapshot.max()
    snapshot = snapshot.astype('uint8')
    image = Image.fromarray(snapshot)
    return image

# generate tensor, because saving image as intermediate data is not efficient
def gen_tensor(p_layer, n_layer, f_layer):
    '''
    Generate a tensor using given matrices.
    Params:
        p_layer: matrix of past layer
        n_layer: matrix of now layer
        f_layer: matrix of future layer
    Return:
        A torch tensor.
    '''
    # create a snapshot
    snapshot = np.zeros([img_size, img_size, 3], dtype='float64')

    # unexpected zones
    left_zones = set()

    # future-Red: 0
    for _, row in f_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 0] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # past-Green: 1
    for _, row in p_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 1] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # now-Blue: 2
    for _, row in n_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 2] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # normalize
    snapshot *= 255 // snapshot.max()
    snapshot = torch.from_numpy(snapshot)
    return snapshot

In [ ]:
# Function that gets a specific layer of snapshot.
def get_channel(image, layer:str):
    '''
    Get a layer of the snapshot.
    Params:
        image: PIL image
        channel: one of R-F,G-P,B-N
    Return:
        single channel image
    '''
    assert layer in ['P', 'N', 'F']
    namedict = {'P': 'G', 'N': 'B', 'F': 'R'}
    chandict = {'R':0, 'G':1, 'B':2}
    template = np.array(image)
    chan = np.zeros([*template.shape], dtype='uint8')
    chan[:,:,chandict[namedict[layer]]] = image.getchannel(namedict[layer])
    chan = Image.fromarray(chan)
    return chan

In [ ]:
# future layer
# first_image.getchannel('R').resize((345,345))
red = get_channel(first_image, 'F')
red = red.resize((345,345))
red_bright = first_image.getchannel('R').resize((345,345))

display(red, red_bright)

In [ ]:
# Past layer
green = get_channel(first_image, 'P')
green = green.resize((345,345))

# Past layer (green) in brightness.
green_bright = first_image.getchannel('G').resize((345,345))
display(green, green_bright)

In [ ]:
# Now layer
# first_image.getchannel('B').resize((345,345))
blue = get_channel(first_image, 'N')
blue = blue.resize((345,345))

# Now layer (blue) in brightness.
blue_bright = first_image.getchannel('B').resize((345,345))

display(blue, blue_bright)

### Using the above procedure, all 4464 tensors can be generated.

**Possible variations:**
+ Weight the value with passenger number
+ Design a method to achieve finer granularity to distinguish trips with much longer time (3X std.)

### 6. Parallel Batch Image Generation

Now let's define a batch processing function to generate tonsors like above according to time interval. For the sake of effitiency, parallel boost is used (i.e. dask). The **computation graphs** of the generator is as follows:

<img src="images/flowgraph_imggen.png" align="left" width="70%">

#### Individual functions:

Create a list of time bounds given the starting point and ending point of a time interval:

In [ ]:
def timesplit(stp: str, etp: str, freq='10min'):
    '''
    Create a DatetimeIndx interval.
    
    Params:
        stp: string, starting time point, first left bound
        etp: string, ending time point, last right bound
        freq: frequency, time interval unit of the splice operation
    The stp and etp must of pattern "yyyy-mm-dd hh:mm:ss", otherwise exception will be raised.
    
    Return:
        A list of time intervals tuples,each item is a tuple of two
        interval(i.e., pandas.core.indexes.datetimes.DatetimeIndex object)
        For example, a possible return could be [(2017-01-01 00:00:00, 2017-01-01 00:10:00),
                                                 (2017-01-01 00:10:00, 2017-01-01 00:20:00)]
    '''
    pattern = re.compile('^([0-9]{4})-([0-1][0-9])-([0-3][0-9])\s([0-1][0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])$')
    if pattern.match(stp) and pattern.match(etp):
        time_bounds = pd.date_range(stp, etp, freq=freq)
        sub_intervals = list(zip(time_bounds[:-1], time_bounds[1:]))
        print(len(time_bounds), len(sub_intervals))
        return sub_intervals
    else:
        raise Exception('Provided time bound is of invalid format.')


In [ ]:
timelist = timesplit('2017-01-01 00:00:00', '2017-02-01 00:00:00')

In [ ]:
# Number of characters needed to save a time pair
sum(list(map(len, ['2017-01-01 00:00:00', '2017-02-01 00:00:00'])))

In [ ]:
# test functions
timelist = timesplit('2017-01-01 00:00:00', '2017-02-01 00:00:00')
print(f'timelist length: {len(timelist)}')

bound = timelist[100]
print(type(bound))
print(bound[0])
print(bound[1])

f_player, f_nlayer, f_flayer = gen_snap_layers(table, bound)
image = gen_image(f_player, f_nlayer, f_flayer)

# image save directory:
visual_path = r'D:\GITHUB\GSPNet\snapshots\visualization'
data_path = r'D:\GITHUB\GSPNet\snapshots\data'

# save image
image.save(f'{data_path}\\d1.bmp')
vimage = image.resize((690,690)) # multiply by factor of 100
vimage.save(f'{visual_path}\\v1.bmp')
vimage

In [ ]:
rdtbt = time.time()
timelist = timesplit('2017-02-01 00:00:00', '2017-03-01 00:00:00')
table = pd.read_csv('dataset/nytaxi_yellow_2017_feb.csv')
print(f'data loaded in {time.time() - rdtbt}')

In [ ]:
table.head()

In [ ]:
start = time.time()
# convert dtype for entire table here:
table['tpep_pickup_datetime'] = pd.to_datetime(table['tpep_pickup_datetime'])
table['tpep_dropoff_datetime'] = pd.to_datetime(table['tpep_dropoff_datetime'])
for i, bound in enumerate(timelist):
    p_layer, n_layer, f_layer = gen_snap_layers(table, bound)
    tensor = gen_tensor(p_layer, n_layer, f_layer)
    torch.save(tensor, f'{data_path}\\nytaxi_yellow_2017_feb_d{i}.pt')
    image = gen_image(p_layer, n_layer, f_layer)
    vimage = image.resize((690,690)) # multiply by factor of 100
    vimage.save(f'{visual_path}\\nytaxi_yellow_2017_feb_v{i}.jpg')
print(f'Image and tensor generation done in {time.time() - start :.2f} seconds.')

In [ ]:
26155 / 60 / 60

## Part 2: Modeling

## Part 3: Prediction & per demand modification of states (to be done)